# Mod 6 Homework Reflections 
# Weeks 5-8

In [3]:
!pip install statsmodels

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 43.8 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [statsmodels] [statsmodels]

[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python3 -m pip install --upgrade pip


In [1]:
import numpy as np
import pandas as pd
import scipy
import matplotlib.pyplot as plt
import statsmodels.formula.api as smf
import statsmodels.api as sm

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy
import statsmodels.api as sm
import seaborn as sns
import networkx as nx
from sklearn.metrics import accuracy_score, roc_auc_score


# Week 5 

1. Draw a diagram for the following negative feedback loop:

Sweating causes body temperature to decrease.  High body temperature causes sweating.

A negative feedback loop means that one thing increases another while the second thing decreases the first.

Remember that we are using directed acyclic graphs where two things cannot directly cause each other.

2. Describe an example of a positive feedback loop.  This means that one things increases another while the second things also increases the first.

3. Draw a diagram for the following situation:

Lightning storms frighten away deer and bears, decreasing their population, and cause flowers to grow, increasing their population.
Bears eat deer, decreasing their population.
Deer eat flowers, decreasing their population.

Write a dataset that simulates this situation.  (Show the code.) Include noise / randomness in all cases.

Identify a backdoor path with one or more confounders for the relationship between deer and flowers.

4. Draw a diagram for a situation of your own invention.  The diagram should include at least four nodes, one confounder, and one collider.  Be sure that it is acyclic (no loops).  Which node would say is most like a treatment (X)?  Which is most like an outcome (Y)?


In [5]:
# Question 3

# Simulating the data 

np.random.seed(42)  
n = 500  # observations

# Exogenous variable: Lightning storms, frequency/intensity 
lightning = np.random.normal(5, 2, n)

# Bears are frightened away by lightning
bears = 100 - 8 * lightning + np.random.normal(0, 5, n)

# Deer are frightened by lightning and eaten by bears
deer = 80 - 5 * lightning - 0.3 * bears + np.random.normal(0, 5, n)

# Flowers: grow more with lightning, eaten by deer
flowers = 50 + 4 * lightning - 0.5 * deer + np.random.normal(0, 5, n)

df = pd.DataFrame({
    "lightning": lightning,
    "bears": bears,
    "deer": deer,
    "flowers": flowers
})

print(df.head())

   lightning      bears       deer    flowers
0   5.993428  56.683461  40.024597  57.853220
1   4.723471  71.759312  39.478018  46.398948
2   6.295377  42.644146  36.028023  53.076502
3   8.046060  38.446368  25.001107  69.666813
4   4.531693  60.493241  42.684678  45.933511


# Week 6

In [1]:
import pandas as pd
import numpy as np
from sklearn.neighbors import NearestNeighbors

df6 = pd.read_csv("homework_6.1.csv")

# Create treated and untreated groups
treated = df6[df6["X"] == 1].reset_index(drop=True)
untreated = df6[df6["X"] == 0].reset_index(drop=True)

# Define nearest-neighbor match function
def nearest_match(source, target):
    nn = NearestNeighbors(n_neighbors=1)
    nn.fit(target[["Z"]])
    distances, indices = nn.kneighbors(source[["Z"]])
    return indices.flatten()

# Match treated to untreated and untreated to treated
treated_to_untreated_idx = nearest_match(treated, untreated)
untreated_to_treated_idx = nearest_match(untreated, treated)

# Generate counterfactual outcomes
treated["Y_cf"] = untreated.loc[treated_to_untreated_idx, "Y"].values
untreated["Y_cf"] = treated.loc[untreated_to_treated_idx, "Y"].values

# Compute treatment effects
treated["TE"] = treated["Y"] - treated["Y_cf"]
untreated["TE"] = treated.loc[untreated_to_treated_idx, "Y"].values - untreated["Y"]

# Compute average treatment effects
ate = (treated["TE"].mean() + untreated["TE"].mean()) / 2
att = treated["TE"].mean()
atu = untreated["TE"].mean()

# Compute Optimal Treatment Effect (max) and MTE using the 90th percentile
optimal_te = untreated["TE"].max()
MTE_90th = np.percentile(untreated["TE"], 90)

# Print results
print(f"Average Treatment Effect (ATE): {ate:.4f}")
print(f"Average Treatment Effect on Treated (ATT): {att:.4f}")
print(f"Average Treatment Effect on Untreated (ATU): {atu:.4f}")
print(f"Naive Optimal Treatment Effect (max): {optimal_te:.4f}")
print(f"Robust Marginal Treatment Effect (90th percentile): {MTE_90th:.4f}")


Average Treatment Effect (ATE): 1.6979
Average Treatment Effect on Treated (ATT): 1.8464
Average Treatment Effect on Untreated (ATU): 1.5495
Naive Optimal Treatment Effect (max): 2.1725
Robust Marginal Treatment Effect (90th percentile): 1.9280


# Week 7

In [2]:
# Week 7, Question 1

import numpy as np
import statsmodels.api as sm

np.random.seed(0)
n = 1000

# Confounder
W = np.random.normal(0, 1, n)

# Treatment
X = W + np.random.normal(0, 1, n)

# Outcome
Y = 2*X + 3*W + np.random.normal(0, 1, n)

# Regress Y and X - no confounder)
model_no_conf = sm.OLS(Y, sm.add_constant(X)).fit()

# Regress Y and X - including confounder
model_full = sm.OLS(Y, sm.add_constant(np.column_stack([X, W]))).fit()

print("True effect of X on Y: 2.0")
print("Without Confounder W:", model_no_conf.params[1])
print("With Confounder W:", model_full.params[1])


True effect of X on Y: 2.0
Without Confounder W: 3.510405682088037
With Confounder W: 1.9981919455639527


In [7]:
# Week 7, Question 2

np.random.seed(0)

p_values = []  

for i in range(1000):
    n = 200  # number of data points

    W = np.random.normal(0, 1, n)
    X = np.random.normal(0, 1, n)
    error = np.random.normal(0, 1, n)

    # true model, coeff of W is 0
    Y = 2 * X + error  

    # regression: including X and W
    XW = sm.add_constant(np.column_stack((X, W)))
    model = sm.OLS(Y, XW).fit()

    # p-value of W is the 3rd value (after constant and X)
    p_w = model.pvalues[2]
    p_values.append(p_w)

# results
min_p = np.min(p_values)
count_sig = np.sum(np.array(p_values) < 0.05)

print("Smallest p-value for W:", round(min_p, 6))
print("Number of times p < 0.05:", count_sig, "out of 1000")

Smallest p-value for W: 0.002831
Number of times p < 0.05: 51 out of 1000


Question 1

Suppose that a process can be modeled via linear regression: 


W = np.random.normal(0, 1, (1000,))
X = W + np.random.normal(0, 1, (1000,)) 
Z = np.random.normal(0, 1, (1000,)) 
Y = X + Z + W + np.random.normal(0, 1, (1000,))
 


Which is closest to the correlation of ﻿X﻿ with the error term in the equation for ﻿Y﻿? 

In [6]:
W = np.random.normal(0, 1, (1000,))
X = W + np.random.normal(0, 1, (1000,)) 
Z = np.random.normal(0, 1, (1000,)) 
Y = X + Z + W + np.random.normal(0, 1, (1000,))

the error term is 
error = Z + W + np.random.normal(0, 1, num)


In [12]:
W = np.random.normal(0, 1, (1000,))
X = W + np.random.normal(0, 1, (1000,))
Z = np.random.normal(0, 1, (1000,))
Y = X + Z + W + np.random.normal(0, 1, (1000,))

# when Y is modeled as depending only on X, the error term = Z + W + random noise
error = Z + W + np.random.normal(0, 1, (1000,))

# compute correlation between X and the error term
corr = np.corrcoef(X, error)[0, 1]
print("Correlation between X and the error term:", corr)

Correlation between X and the error term: 0.40335501644287275


Question 2


If ﻿Y﻿ is written as depending on ﻿X﻿ and ﻿Z﻿ only, ﻿W﻿ is part of the error term. Which, then, is closest to the expected correlation of ﻿X﻿ with the error term in the equation for ﻿Y﻿? 



In [8]:

W = np.random.normal(0, 1, (1000,))
X = W + np.random.normal(0, 1, (1000,))
Z = np.random.normal(0, 1, (1000,))
Y = X + Z + W + np.random.normal(0, 1, (1000,))

# since Y depends on X and Z only, the error term contains W + random noise
error = W + np.random.normal(0, 1, (1000,))

# compute correlation between X and the error term
corr = np.corrcoef(X, error)[0, 1]
print("Correlation between X and the error term:", corr)


Correlation between X and the error term: 0.48407444829165097


Question 3

In the data frame for homework_7.1.csv, control for W by regressing ﻿Y﻿ on ﻿X﻿ and ﻿Z﻿ at the following constant values of ﻿W﻿: -1, 0, and 1. (You cannot literally use a constant value of ﻿W﻿, of course, or you will have only one data point! How will you manage this?) The question is: Is the coefficient of ﻿X﻿  

Option A
increasing

Option B
decreasing

Option C
staying about the same (say, within 0.2 or so) as ﻿W﻿ increases? 

In [13]:
df7 = pd.read_csv("homework_7.1.csv")
df7.head()

,Unnamed: 0,X,W,Z,Y
0,0,1.137055,1.221768,0.327829,1.944532
1,1,-0.112905,0.465835,0.599650,0.655514
2,2,2.077755,1.795414,-0.063393,5.934411
3,3,0.456373,-0.512159,1.177413,-0.188064
4,4,-1.012402,0.080002,-0.275697,-0.533775


In [16]:
# Subset the data where W is close to -1
df_w_neg1 = df7[(df7['W'] > -1.5) & (df7['W'] < -0.5)]
model_neg1 = sm.OLS(df_w_neg1['Y'], sm.add_constant(df_w_neg1[['X', 'Z']])).fit()
print("When W is around -1, coefficient of X =", model_neg1.params['X'])

# W around 0
df_w_0 = df7[(df7['W'] > -0.5) & (df7['W'] < 0.5)]
model_0 = sm.OLS(df_w_0['Y'], sm.add_constant(df_w_0[['X', 'Z']])).fit()
print("When W is around 0, coefficient of X =", model_0.params['X'])

# W around 1
df_w_1 = df7[(df7['W'] > 0.5) & (df7['W'] < 1.5)]
model_1 = sm.OLS(df_w_1['Y'], sm.add_constant(df_w_1[['X', 'Z']])).fit()
print("When W is around 1, coefficient of X =", model_1.params['X'])


When W is around -1, coefficient of X = 0.990090408694134
When W is around 0, coefficient of X = 1.4859822514480032
When W is around 1, coefficient of X = 1.9936504417092331


Question 4


def make_error(corr_const, num): 

 err = list() 

    prev = np.random.normal(0, 1) 

 for n in range(num): 

    prev = corr_const * prev + (1 - corr_const) * np.random.normal(0, 1) 

    err.append(prev) 

return np.array(err) 



Create a linear regression model that uses this function as the error for both (a) the treatment, ﻿X﻿, and (b) the outcome, ﻿Y﻿. (You can use random normal error for any other covariates, if you have them.) 



As corr_const increases from 0.2 to 0.5 to 0.8, find (i) the standard deviation of the estimate of the ﻿X﻿ coefficient over many trials, and (ii) the mean of the standard error estimate of the ﻿X﻿ coefficient over many trials. 



When corr_const increases, then: 



Hint: don't forget to include an intercept in your regression

Option A
(i) and (ii) remain about the same

Option B
The ratio (i) / (ii) decreases

Option C
(i) and (ii) differ, but their ratio remains about the same

Option D
The ratio (i) / (ii) increases



In [17]:
# Function to make correlated errors
def make_error(corr_const, num):
    err = []
    prev = np.random.normal(0, 1)
    for i in range(num):
        prev = corr_const * prev + (1 - corr_const) * np.random.normal(0, 1)
        err.append(prev)
    return np.array(err)

corr_levels = [0.2, 0.5, 0.8]

# To store the results
results = []

# Fix random seed so we can reproduce results
np.random.seed(42)

# For each correlation level
for corr_const in corr_levels:
    all_betas = []
    all_ses = []
    
    # Run many simulations
    for i in range(100):  # 100 trials
        n = 1000
        
        # Create errors for X and Y
        eX = make_error(corr_const, n)
        eY = make_error(corr_const, n)
        
        # Generate data
        Z = np.random.normal(0, 1, n)
        X = 0.5 * Z + eX
        Y = 1.0 * X + 0.3 * Z + eY
        
        # Run regression: Y ~ X + Z
        X_matrix = sm.add_constant(np.column_stack([X, Z]))
        model = sm.OLS(Y, X_matrix).fit()
        
        # Save the coefficient for X and its standard error
        all_betas.append(model.params[1])  # X coefficient
        all_ses.append(model.bse[1])       # X standard error
    
    # Compute (i) and (ii)
    std_dev_of_betas = np.std(all_betas)
    mean_se = np.mean(all_ses)
    ratio = std_dev_of_betas / mean_se
    
    # Save the results
    results.append([corr_const, std_dev_of_betas, mean_se, ratio])

# Show results
results_df = pd.DataFrame(results, columns=["corr_const", "(i) std of β_X", "(ii) mean SE", "ratio (i)/(ii)"])
print(results_df)


   corr_const  (i) std of β_X  (ii) mean SE  ratio (i)/(ii)
0         0.2        0.029915      0.031531        0.948757
1         0.5        0.041477      0.031510        1.316324
2         0.8        0.067064      0.031621        2.120874
